## LSSTCam Astrometry learning with GAIA

Craig Lage - 19-May-25

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lsst.daf.butler import Butler
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.geom import SpherePoint,Angle,Extent2I,Box2I,Extent2D,Point2D, Point2I
from astroquery.gaia import Gaia
from lsst.summit.utils.plotting import plot
from lsst.obs.lsst import LsstCam
from lsst.obs.lsst.cameraTransforms import LsstCameraTransforms

In [ ]:
def get_stars_focal_plane_gaia(camera, img_detName, ref_detName, stamp_centerX, stamp_centerY, rows, cols, ref_wcs):
    # find the ra,dec of the CCD center
    img_detector = camera[img_detName]
    img_bbox = img_detector.getBBox()
    nx,ny = img_bbox.getDimensions()
    lct = LsstCameraTransforms(camera,img_detName)
    llfpX, llfpY = lct.ccdPixelToFocalMm(0, 0, img_detName)
    llfpX = llfpX * 100 + int(nx/2) # Convert to pixels and recenter
    llfpY = llfpY * 100 + int(ny/2) # Convert to pixels and recenter
    urfpX, urfpY = lct.ccdPixelToFocalMm(nx, ny, img_detName)
    urfpX = urfpX * 100 + int(nx/2) # Convert to pixels and recenter
    urfpY = urfpY * 100 + int(ny/2) # Convert to pixels and recenter
    if llfpX > urfpX:
        llfpX, urfpX = urfpX, llfpX
    if llfpY > urfpY:
        llfpY, urfpY = urfpY, llfpY

    ref_detector = camera[ref_detName]
    ref_bbox = ref_detector.getBBox()
    ref_llfpX, ref_llfpY = lct.ccdPixelToFocalMm(0, 0, ref_detName)
    ref_llfpX = ref_llfpX * 100 + int(nx/2) # Convert to pixels and recenter
    ref_llfpY = ref_llfpY * 100 + int(ny/2) # Convert to pixels and recenter

    ccd_bbox = Box2I(Point2I(llfpX - ref_llfpX + stamp_centerX - cols/2, llfpY - ref_llfpY + stamp_centerY - rows/2), 
                     Point2I(llfpX - ref_llfpX + stamp_centerX + cols/2, llfpY - ref_llfpY + stamp_centerY + rows/2))

    Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source" 

    
    ra_stamp,dec_stamp = ref_wcs.pixelToSky(stamp_centerX - ref_llfpX + llfpX, stamp_centerY - ref_llfpY + llfpY)
    print("In Gaia", stamp_centerX - ref_llfpX + llfpX, stamp_centerY - ref_llfpY + llfpY)
    rad = u.Quantity(0.30, u.deg) # 0.04 degrees is enough to cover a 400x400 stamp
    # Could be reduced for smaller stamps.
    cols = ['ra', 'dec', 'phot_g_mean_mag', 'designation']
    
    coord = SkyCoord(ra=ra_stamp.asDegrees(), dec=dec_stamp.asDegrees(), unit=(u.degree, u.degree), frame='icrs')
    Gaia.ROW_LIMIT = 50000
    r = Gaia.query_object_async(coordinate=coord, radius=rad, columns=cols)
    
    gaia_table = r.to_pandas()
    # using the wcs to locate the stars inside the CCD minus npixedge
    ccdx,ccdy = ref_wcs.skyToPixelArray(gaia_table['ra'],gaia_table['dec'], degrees=True)
    inCCD = ccd_bbox.contains(ccdx,ccdy)

    # fill with CCD pixel x,y
   
    gaia_table['ccdx'] = ccdx - llfpX + ref_llfpX
    gaia_table['ccdy'] = ccdy - llfpY + ref_llfpY
    gaia_table['inCCD'] = inCCD

    return gaia_table


In [ ]:
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation',
                                              'LSSTCam/runs/nightlyValidation/20250425/w_2025_17/DM-50157'])
instrument = 'LSSTCam'

## The cell below takes the WCS from one imaging CCD and applies it to another imaging CCD.  Does not work for guider CCDs

In [ ]:
expId = 2025071200535

mag_limit = 13.0
ref_detName = 'R14_S12' # detector 68
img_detName = 'R10_S10' #detector 30

camera = LsstCam.getCamera()
ref_detector = camera[ref_detName]
ref_calexp = butler.get('preliminary_visit_image', detector=ref_detector.getId(), visit=expId)
ref_wcs = ref_calexp.getWcs()
ref_bbox = ref_detector.getBBox()
ref_center = ref_wcs.pixelToSky(ref_bbox.centerX, ref_bbox.centerY)
ref_ra_center = ref_center.getRa().asDegrees()
ref_dec_center = ref_center.getDec().asDegrees()


img_detector = camera[img_detName]
img_calexp = butler.get('preliminary_visit_image', detector=img_detector.getId(), visit=expId)
img_raw = butler.get('raw', detector=img_detector.getId(), exposure=expId)
#img_wcs = img_calexp.getWcs()
img_wcs = img_raw.getWcs()
img_bbox = img_detector.getBBox()
img_center = img_wcs.pixelToSky(img_bbox.centerX, img_bbox.centerY)
img_ra_center = img_center.getRa()
img_dec_center = img_center.getDec()

cat_select = get_stars_focal_plane_gaia(camera, img_detName, ref_detName, 2048, 2000, 4000, 4000, ref_wcs)
xs = []
ys = []
mags = []
designations = []
for i in range(len(cat_select)):
    x = cat_select['ccdx'][i]
    y = cat_select['ccdy'][i]
    if cat_select['inCCD'][i] and cat_select['phot_g_mean_mag'][i] < mag_limit:
        xs.append(x)
        ys.append(y)
        mags.append(f"{cat_select['phot_g_mean_mag'][i]:.1f}")
        designations.append(f"{cat_select['designation'][i]}")


# Now plot the data with matplotlib
%matplotlib inline
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

fig = plt.figure(figsize=(16,16))

myPlot = plot(img_calexp, stretch='ccs', figure=fig)
ax = myPlot.get_axes()[0]
ax.scatter(xs, ys\
            ,facecolors='none', edgecolors='magenta', s=200, lw=2)
for x, y, mag in zip(xs, ys, mags):
    ax.text(x+50,y, mag, color='magenta', fontsize=12, fontweight='bold')
ax.set_title(f"{expId} {img_detName}, WCS from {ref_detName}")
plt.savefig(f"/home/c/cslage/u/LSSTCam/images/WCS_{expId}_{img_detName}_WCS_From_{ref_detName}.png")

# The cell below prints out the cells brighter than mag_limit, with their location and GAIA designation.

In [ ]:
for x, y, mag, designation in zip(xs, ys, mags, designations):
    print(x, y, mag, designation)
